In [1]:
# Install required packages.
import os
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Install the CPU version.
device = torch.device("cpu")
# !pip install --pre dgl -f https://data.dgl.ai/wheels-test/repo.html

try:
    import dgl
    import dgl.graphbolt as gb
    installed = True
except ImportError as error:
    installed = False
    print(error)
print("DGL installed!" if installed else "DGL not found!")

/Users/kushaldsouza/miniconda3/envs/dgl/lib/python3.12/site-packages/torchdata-0.8.0-py3.12-macosx-11.0-arm64.egg/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


DGL installed!


In [2]:
# dataset = gb.BuiltinDataset("ogbl-citation2").load()

In [3]:
# print(dataset.tasks[0].test_set)
# print(dataset.graph)

In [4]:
# import torch
# print(np.random.randint(0, 5, size=(5, 2)))
# import numpy as np
# personNodesWithLinks = [0, 1, 3, 4]
# personNodesNp = np.array(personNodesWithLinks)
# print(personNodesNp)

In [5]:
import os
import dgl
import torch
import numpy as np
import pandas as pd

# Define the graph structure
# Define directories
base_dir = './sample_dataset'
os.makedirs(base_dir, exist_ok=True)
personNodes = [0, 1, 2, 3, 4]
personNodesWithLinks = [0, 1, 3, 4]
personNodesNp = np.array(personNodesWithLinks)
itemNodes = [0, 1, 2, 3]
foodNodes = [0, 1, 2, 3]
personItemEdges = [[0, 0], [1, 1], [2, 3], [1, 4]]
personItemTotalEdges = [[[0, 0], [0,1]], [[1, 1], [1,2]], [[3, 2], [3, 1]], [[4, 1], [4,3]]]
personItemEdgesNp = np.array(personItemEdges)
personItemTotalEdgesNp = np.array(personItemTotalEdges)
# df = pd.DataFrame(personItemEdgesNp)
# df.to_csv(os.path.join(base_dir, "person_item_edges.csv"), index=False, header=False)
print(f"[item:liked_by:person] edges: {personItemEdgesNp.T}\n")
np.save(os.path.join(base_dir, "person_item_edges.npy"), personItemEdgesNp)
np.save(os.path.join(base_dir, "person_item_edges_t.npy"), personItemEdgesNp.T)
np.save(os.path.join(base_dir, "person_nodes.npy"), personNodesNp)
np.save(os.path.join(base_dir, "person_item_edges_total.npy"), personItemTotalEdgesNp)

personFoodEdges = ([0, 0], [2, 1], [1, 2], [3, 4])
personFoodEdgesNp = np.stack(personFoodEdges)
# df = pd.DataFrame(personFoodEdgesNp)
# df.to_csv(os.path.join(base_dir, "person_food_edges.csv"), index=False, header=False)
np.save(os.path.join(base_dir, "person_food_edges.npy"), personFoodEdgesNp)
np.save(os.path.join(base_dir, "person_food_edges_t.npy"), personFoodEdgesNp.T)

# Add features to nodes (dummy features for this example)
features = torch.eye(len(personNodes))  # Identity matrix as features

# Save node features
torch.save(features, os.path.join(base_dir, "node_p_features.pt"))
torch.save(features * 2, os.path.join(base_dir, "node_i_features.pt"))
torch.save(features * 3, os.path.join(base_dir, "node_f_features.pt"))

# Create the metadata.yaml file
yaml_content = f"""
dataset_name: sample_graph
graph:
  nodes:
    - type: person
      num: {len(personNodes)}
    - type: item
      num: {len(itemNodes)}
    - type: food
      num: {len(foodNodes)}
  edges:
    - type: "item:liked_by:person"
      format: numpy
      path: ./person_item_edges_t.npy
    - type: "food:liked_by:person"
      format: numpy
      path: ./person_food_edges_t.npy
feature_data:
  - domain: node
    type: person
    name: p_feat
    format: torch
    in_memory: false
    path: node_p_features.pt
  - domain: node
    type: item
    name: i_feat
    format: torch
    in_memory: false
    path: node_i_features.pt
  - domain: node
    type: food
    name: f_feat
    format: torch
    in_memory: false
    path: node_f_features.pt
tasks:
  - name: link_prediction
    train_set:
      - type: "item:liked_by:person"
        data:
          - name: seeds
            format: numpy
            in_memory: false
            path: ./person_item_edges.npy
          # - name: seeds_with_negative
          #   format: numpy
          #   in_memory: false
          #   path: ./person_item_edges_total.npy
        
"""

# Save metadata.yaml
with open(os.path.join(base_dir, "metadata.yaml"), "w") as f:
    f.write(yaml_content)


[person:like:item] edges: [[0 1 2 1]
 [0 1 3 4]]



In [6]:
# Load the dataset to test
base_dir = './sample_dataset'
print(base_dir)
dataset = gb.OnDiskDataset(base_dir, force_preprocess = True).load()
# print(f"Loaded graph: {graph}")

# features = dataset.feature
# print(f"Loaded feature store: {features}")

# tasks = dataset.tasks
# lp_task = tasks[0]
# print(f"Loaded link prediction task: {lp_task}")

./sample_dataset
The on-disk dataset is re-preprocessing, so the existing preprocessed dataset has been removed.
Start to preprocess the on-disk dataset.
Finish preprocessing the on-disk dataset.


In [7]:
device = torch.device("cpu")
itemset = dataset.tasks[0].train_set
graph = dataset.graph.to(device)
feature = dataset.feature.to(device)
print(itemset)
print(graph)
print(feature)

HeteroItemSet(
    itemsets={'item:liked_by:person': ItemSet(
                 items=(tensor([[0, 0],
                     [1, 1],
                     [2, 3],
                     [1, 4]], dtype=torch.int32),),
                 names=('seeds',),
             )},
    names=('seeds',),
)
FusedCSCSamplingGraph(csc_indptr=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 5, 6, 8], dtype=torch.int32),
                      indices=tensor([0, 4, 2, 5, 1, 6, 3, 5], dtype=torch.int32),
                      total_num_nodes=13, num_edges={'food:liked_by:person': 4, 'item:liked_by:person': 4},
                      node_type_offset=tensor([ 0,  4,  8, 13], dtype=torch.int32),
                      type_per_edge=tensor([0, 1, 0, 1, 0, 1, 0, 1], dtype=torch.uint8),
                      node_type_to_id={'food': 0, 'item': 1, 'person': 2},
                      edge_type_to_id={'food:liked_by:person': 0, 'item:liked_by:person': 1},)
TorchBasedFeatureStore(
    {(<OnDiskFeatureDataDomain.NODE: 'node'>, 'per

In [8]:
# def minibatcher(batch, names):
#     print(batch)
#     print(names)
#     return gb.MiniBatch(seeds=batch.seeds)

# sampler = gb.ItemSampler(
#         itemset,
#         batch_size=1,
#         # minibatcher=minibatcher,
#         shuffle=True,
#     )

In [9]:
# next(iter(sampler))
# for minibatch in sampler:
#     print(minibatch)

In [10]:
# sampler = sampler.sample_uniform_negative(graph, 1)

In [11]:
# for minibatch in sampler:
#     # print(minibatch)
    # print(minibatch.seeds)

In [12]:
# from functools import partial
# sampler = sampler.sample_neighbor(
#         graph, [-1]
#     )
# sampler = sampler.transform(partial(gb.exclude_seed_edges, include_reverse_edges=False))


In [13]:
# next(iter(sampler))
# for minibatch in sampler:
#     print(minibatch)

In [14]:
# from functools import partial
# sampler = sampler.transform(
#             partial(gb.exclude_seed_edges, include_reverse_edges=False)
#         )

In [15]:
# for minibatch in sampler:
#     print(minibatch)

In [23]:
datapipe = gb.ItemSampler(itemset, batch_size=2, shuffle=True)
datapipe = datapipe.copy_to(device)
datapipe = datapipe.sample_neighbor(graph, [-1, -1, -1])
datapipe = datapipe.fetch_feature(feature, node_feature_keys={"person": ["p_feat"], "item": ["i_feat"], "food": ["f_feat"]})

In [21]:
def create_train_dataloader():
    datapipe = gb.ItemSampler(itemset, batch_size=1, shuffle=True)
    datapipe = datapipe.copy_to(device)
    datapipe = datapipe.sample_neighbor(graph, [-1, -1, -1])
    datapipe = datapipe.fetch_feature(feature, node_feature_keys={"person": ["p_feat"], "item": ["i_feat"], "food": ["f_feat"]})
    return gb.DataLoader(datapipe)

In [22]:
for step, data in enumerate(create_train_dataloader()):
    print("\n\n\n")
    # print(data.seeds)
    # print(data.blocks[0])
    print(data.seeds)
    print(data.blocks[0].edges(etype=('food', 'liked_by', 'person')))
    print(data.blocks[0].edges(etype=('item', 'liked_by', 'person')))
    print(data.blocks[1].edges(etype=('food', 'liked_by', 'person')))
    print(data.blocks[1].edges(etype=('item', 'liked_by', 'person')))
    print(data.blocks[2].edges(etype=('food', 'liked_by', 'person')))
    print(data.blocks[2].edges(etype=('item', 'liked_by', 'person')))
    # src, dest = data.blocks[1].edges(etype=('person', 'like', 'item'))
    # print(src.numpy())
    # print(dest.numpy())
    # print(np.stack((src.numpy(), dest.numpy())))
    # print(data.blocks[1])
    # print(data.blocks[0].canonical_etypes)
    # print(data)





{'item:liked_by:person': tensor([[0, 0]], dtype=torch.int32)}
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([0], dtype=torch.int32), tensor([0], dtype=torch.int32))
(tensor([0], dtype=torch.int32), tensor([0], dtype=torch.int32))




{'item:liked_by:person': tensor([[2, 3]], dtype=torch.int32)}
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([], dtype=torch.int32), tensor([], dtype=torch.int32))
(tensor([0], dtype=torch.int32), tensor([0], dtype=torch.int32))




{'item:liked_by:person': tensor([[1, 4]], dtype=torch.int32)}
(tensor([], dtype=torch.int32), tensor([